In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
import gradio as gr 

In [3]:
# Load environment variables from .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif api_key[:8]!= "sk-proj-":
    print("An API Key was found, but it doesn't start with 'sk-proj-', please check you're using right api key")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
# Generic system message

system_message = "You are a helpful assistant"

In [5]:
openai = OpenAI()

In [6]:
# Lets wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        max_tokens= 500,
    )
    return response.choices[0].message.content

In [7]:
message_gpt("What is today's date?")     # still not great with current events

"Today's date is October 3, 2023."

Lets build the User Interface with Gradio

In [14]:
# simple function

def shout(text):
    print(f"The function shout has been called with the text : {text}")
    return text.upper()

The function shout has been called with the text : {text}
The function shout has been called with the text : {text}
The function shout has been called with the text : {text}


In [9]:
shout('hello, welcome to gradio!')

'HELLO, WELCOME TO GRADIO!'

Gradio can take single as well as multiple inputs and outputs 

In [10]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox")
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never")     #not to create flagged folder
view. launch(share=True)

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


The user interface is served off as a public server (url) but the function is running on your local machine, which is amazing with gradio

In [15]:
view = gr.Interface(
    fn = shout,
    inputs= [gr.Textbox(label="Your message :", lines = 6)],
    outputs= [gr.Textbox(label="Response :", lines=8)],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


The function shout has been called with the text : hey


Now will assign gpt input prompt as a input function to gradio and the gpt response will come as a output of gradio interface

In [16]:
view = gr.Interface(
    fn = message_gpt,
    inputs= [gr.Textbox(label="Your message :", lines = 6)],
    outputs= [gr.Textbox(label="Response :", lines=8)],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Will try to show the response generated in a Markdown

In [18]:
system_message = "You are a helpful assistant that responds in markdown."

view = gr.Interface(
    fn = message_gpt,
    inputs= [gr.Textbox(label="Your message :")],
    outputs= [gr.Markdown(label="Response :")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Lets stream the response with gradio. Take a note that while streaming the output response with gradio we can not stream back chunk by chunk rather we stream back the entire commulative result with the yield generator

In [19]:
# Lets wrap a call to GPT-4o-mini in a simple function

def stream_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    stream_response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        max_tokens= 256,
        stream= True
    )
    result = ""
    for chunk in stream_response:
        result += chunk.choices[0].delta.content or ""
        yield result


In [20]:
view = gr.Interface(
    fn = stream_gpt,
    inputs= [gr.Textbox(label="Your message :")],
    outputs= [gr.Markdown(label="Response :")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Lets use multiple models and give user a chance to select the model to respond for its conversations

In [27]:
import ollama

def stream_ollama(prompt):
    messages = [
        {"role": "system", "content":system_message},
        {"role": "user", "content": prompt}
    ]
    
    stream_response = ollama.chat(
        model="llama3.2",  # Change to your preferred model (e.g., 'mistral', 'gemma', etc.)
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream_response:
        result += chunk['message']['content']
        yield result


In [25]:
for output in stream_ollama("Explain deep learning in simple terms."):
    print(output, end='', flush=True)


DeepDeep learningDeep learning isDeep learning is aDeep learning is a typeDeep learning is a type ofDeep learning is a type of machineDeep learning is a type of machine learningDeep learning is a type of machine learning thatDeep learning is a type of machine learning that usesDeep learning is a type of machine learning that uses artificialDeep learning is a type of machine learning that uses artificial neuralDeep learning is a type of machine learning that uses artificial neural networksDeep learning is a type of machine learning that uses artificial neural networks toDeep learning is a type of machine learning that uses artificial neural networks to makeDeep learning is a type of machine learning that uses artificial neural networks to make predictionsDeep learning is a type of machine learning that uses artificial neural networks to make predictions orDeep learning is a type of machine learning that uses artificial neural networks to make predictions or decisionsDeep learning is a t

In [26]:
view = gr.Interface(
    fn = stream_ollama,
    inputs= [gr.Textbox(label="Your message :")],
    outputs= [gr.Markdown(label="Response :")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [31]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "OLLAMA":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [32]:
view = gr.Interface(
    fn = stream_model,
    inputs= [gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "OLLAMA"], label='Select model')],
    outputs= [gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Build Company Brochure Generator

In [35]:
# A class to represent Webpage

class Website:
    url : str
    title : str
    text : str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title : \n {self.title} \n Webpage Contents : \n {self.text} \n\n"

In [36]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [37]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is the Landing page : \n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "OLLAMA":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    for response in result:
        yield response

In [38]:
view = gr.Interface(
    fn = stream_brochure,
    inputs= [
        gr.Textbox(label="Company name :"),
        gr.Textbox(label="Landing page URL: "),
        gr.Dropdown(["GPT", "OLLAMA"], label="Select Model")
    ],
    outputs= [gr.Markdown(label="Brochure: ")],
    allow_flagging= "never"
)
view.launch()

c:\Users\KajalMore\VSCODE\self_learning\env\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
